# $N$ QPU VQE experiment

This notebook goes through the running of the parallel VQE algorithm on $N$ QPUs using Interlin-q. This notebook differs from the manual version in the manner it calculates the expectation value. Here, the controller host sends the hamiltonian to the computing hosts, and then asks it for the expectation value later through interlin-q API. This approach should scale up with the increasing number of computing hosts.

This notebook also won't include the 'single-step' implementation, until the optimization part.

## Step 1: Import libraries.

First we import all the necessary libraries. Interlin-q is build using the python framework [QuNetSim](https://arxiv.org/abs/2003.06397) which is a python software framework that can be used to simulate quantum networks up to the network layer. We also need PennyLane's chemistry library for decomposing the Hamiltonian. We would also use PennyLane for the optimiser component

In [1]:
%load_ext autoreload

%autoreload 2

# Basic Libraries
import sys
from pennylane import numpy as np
sys.path.append("../../")

# QuNetSim Components
from qunetsim.components import Network
from qunetsim.objects import Logger
from qunetsim.backends.qutip_backend import QuTipBackend

# Interlin-q Components
from interlinq import (ControllerHost, Constants, Clock,
Circuit, Layer, ComputingHost, Operation)

# Extra needed components
from pennylane import GradientDescentOptimizer
from hamiltonian_decomposition import decompose

Logger.DISABLED = False

## Step 2: Decompose the Hamiltonian.

In [2]:
geometry = 'h2.xyz'
charge = 0
multiplicity = 1
basis_set = 'sto-3g'
name = 'h2'

In [3]:
coefficients, observables, qubit_num = decompose(name, geometry, charge, multiplicity, basis_set)
terms = list(zip(coefficients, observables))

terms

[(-0.04207897647782276, [('Identity', 0)]),
 (0.17771287465139946, [('PauliZ', 0)]),
 (0.1777128746513994, [('PauliZ', 1)]),
 (-0.24274280513140462, [('PauliZ', 2)]),
 (-0.24274280513140462, [('PauliZ', 3)]),
 (0.17059738328801052, [('PauliZ', 0), ('PauliZ', 1)]),
 (0.04475014401535161,
  [('PauliY', 0), ('PauliX', 1), ('PauliX', 2), ('PauliY', 3)]),
 (-0.04475014401535161,
  [('PauliY', 0), ('PauliY', 1), ('PauliX', 2), ('PauliX', 3)]),
 (-0.04475014401535161,
  [('PauliX', 0), ('PauliX', 1), ('PauliY', 2), ('PauliY', 3)]),
 (0.04475014401535161,
  [('PauliX', 0), ('PauliY', 1), ('PauliY', 2), ('PauliX', 3)]),
 (0.12293305056183798, [('PauliZ', 0), ('PauliZ', 2)]),
 (0.1676831945771896, [('PauliZ', 0), ('PauliZ', 3)]),
 (0.1676831945771896, [('PauliZ', 1), ('PauliZ', 2)]),
 (0.12293305056183798, [('PauliZ', 1), ('PauliZ', 3)]),
 (0.17627640804319591, [('PauliZ', 2), ('PauliZ', 3)])]

## Step 3: Prepare Circuit for Given Parameters

The circuit can be prepared using two different ways: either as one circuit, or several circuits run sequentially. The former approach is simpler and generally better for the optimisation function. The latter is better for debugging and for dynamic components of a quantum circuit (i.e. circuits that have a lot of changing operations).

### Main Blocks

In [4]:
def rotational_gate(params):
    phi, theta, omega = params
    cos = np.cos(theta / 2)
    sin = np.sin(theta / 2)
    
    res = np.array([[np.exp(-1j * (phi + omega) / 2) * cos, -np.exp(1j * (phi - omega) / 2) * sin], 
                     [np.exp(-1j * (phi - omega) / 2) * sin, np.exp(1j * (phi + omega) / 2) * cos]])
    
    return res.unwrap()

In [5]:
def initialisation_operations(q_map):
    computing_host_ids = list(q_map.keys())
    
    ops = []
    for host_id in computing_host_ids:
        op = Operation(
            name=Constants.PREPARE_QUBITS,
            qids=q_map[host_id],
            computing_host_ids=[host_id])
        ops.append(op)
    
        # Prepare the qubits on the computing host
        op = Operation(
            name=Constants.SINGLE,
            qids=[q_map[host_id][0]],
            gate=Operation.X,
            computing_host_ids=[host_id])
        ops.append(op)

        op = Operation(
            name=Constants.SINGLE,
            qids=[q_map[host_id][1]],
            gate=Operation.X,
            computing_host_ids=[host_id])
        ops.append(op)
    
    return [Layer(ops)]

In [6]:
def ansatz_operations(q_map, parameters):
    computing_host_ids = list(q_map.keys())
    
    layers = []
    
    ops = []
    for host_id in computing_host_ids:
        for i in range(len(q_map[host_id])):
            op = Operation(
                name=Constants.SINGLE,
                qids=[q_map[host_id][i]],
                gate=Operation.CUSTOM,
                gate_param=rotational_gate(parameters[i]),
                computing_host_ids=[host_id])

            ops.append(op)

    layers.append(Layer(ops))
    
    ops = []
    for host_id in computing_host_ids:
        op = Operation(
            name=Constants.TWO_QUBIT,
            qids=[q_map[host_id][2], q_map[host_id][3]],
            gate=Operation.CNOT,
            computing_host_ids=[host_id])
        
        ops.append(op)

        op = Operation(
            name=Constants.TWO_QUBIT,
            qids=[q_map[host_id][2], q_map[host_id][0]],
            gate=Operation.CNOT,
            computing_host_ids=[host_id])

        ops.append(op)

        op = Operation(
            name=Constants.TWO_QUBIT,
            qids=[q_map[host_id][3], q_map[host_id][1]],
            gate=Operation.CNOT,
            computing_host_ids=[host_id])

        ops.append(op)
    
    layers.append(Layer(ops))
        
    return layers

In [7]:
def dispatch_hamiltonian_schedules(host, q_map):
        """
        Send the assigned schedules to the computing hosts
        """

        # Create the correct operations
        ops = []
        computing_hosts_ids = list(q_map.keys())
        
        for computing_host_id in computing_hosts_ids:    
            op = Operation(
                name=Constants.REC_HAMILTON,
                computing_host_ids=[computing_host_id],
                hamiltonian=host.term_assignment[computing_host_id])
            ops.append(op)

        layers = [Layer(ops)]

        return layers

In [8]:
    def ask_for_expectation_values(q_map):
        """
        Queries the computing hosts for their expectation values
        """

        # Create the correct operations
        ops = []
        computing_hosts_ids = list(q_map.keys())
        
        for computing_host_id in computing_hosts_ids:    
            op = Operation(
                name=Constants.SEND_EXP,
                computing_host_ids=[computing_host_id])
            ops.append(op)

        layers = [Layer(ops)]

        return layers

### The Protocols

#### The Controller Protocols 

In [9]:
def prepare_qubits_ansatz(q_map, parameters): 
    circuit = Circuit(q_map, initialisation_operations(q_map) + ansatz_operations(q_map, parameters))
    return circuit

In [10]:
def controller_host_protocol_preparation_ansatz(host, q_map, params, terms):
    """
    Protocol for the controller host
    """
    host.schedule_expectation_terms(terms, q_map)
    
    host.generate_and_send_schedules(prepare_qubits_ansatz(q_map, params))

In [11]:
def controller_host_protocol_expectation_schedule(host, q_map, terms):
    """
    Protocol for the controller host
    """
    circuit = Circuit(q_map, dispatch_hamiltonian_schedules(host, q_map))
    
    host.generate_and_send_schedules(circuit)

In [12]:
def controller_host_protocol_expectation_collection(host, q_map, terms):
    """
    Protocol for the controller host
    """
    circuit = Circuit(q_map, ask_for_expectation_values(q_map))
    
    host.generate_and_send_schedules(circuit)
    
    host.receive_results()

#### The Computing Protocols

In [13]:
def computing_host_protocol(host):
    host.receive_schedule()

In [14]:
def computing_host_protocol_send_results(host):
    host.receive_schedule()
    
    host.send_results('expectation')

## Step 4: Run the circuit and get the Expectation Value

In [15]:
def init_network():
    network = Network.get_instance()
    network.delay = 0
    network.start()

    clock = Clock()

    qutip = QuTipBackend()

    controller_host = ControllerHost(
        host_id="host_1",
        clock=clock, 
        backend=qutip
    )

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=3,
        num_qubits_per_host=4)
    controller_host.start()

    network.add_hosts([controller_host])
    network.add_hosts(computing_hosts)
    
    return clock, controller_host, computing_hosts, q_map

In [16]:
params = np.random.uniform(0, 2 * np.pi, size=(4,3))

### Running the circuit

In [17]:
list_hosts = list(Network.get_instance().ARP.keys())

for key in list_hosts:
    Network.get_instance().remove_host(Network.get_instance().get_host(key))

In [18]:
clock, controller_host, computing_hosts, q_map = init_network()

INFO:qu_net_sim:Host QPU_0 started processing
INFO:qu_net_sim:Host QPU_1 started processing
INFO:qu_net_sim:Host QPU_2 started processing
INFO:qu_net_sim:Host host_1 started processing


In [19]:
t1 = controller_host.run_protocol(
    controller_host_protocol_preparation_ansatz,
    (q_map, params, terms))

threads = []

for host in computing_hosts:
    threads.append(host.run_protocol(computing_host_protocol))

t1.join()
for thread in threads:
    thread.join()

INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:1 from QPU_2 to host_1
INFO:qu_net_sim:sending ACK:1 from QPU_1 to host_1
INFO:qu_net_sim:sending ACK:1 from QPU_0 to host_1
INFO:qu_net_sim:QPU_2 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "custom_gate", "gate_param": [[[-0.08994573271629325, 0.1742142378941241], [-0.9688706407057829, -0.15115834763407704]], [[0.9688706407057829, -0.15115834763407704]

INFO:qu_net_sim:QPU_1 sends CLASSICAL to host_1 with sequence 0
INFO:qu_net_sim:host_1 received ACK from QPU_2 with sequence number 0
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "custom_gate", "gate_param": [[[-0.08994573271629325, 0.1742142378941241], [-0.9688706407057829, -0.15115834763407704]], [[0.9688706407057829, -0.15115834763407704], [-0.08994573271629325, -0.1742142378941241]]], "computing_host_i

INFO:qu_net_sim:host_1 received ACK with sequence number 0
INFO:qu_net_sim:QPU_2 received ACK from host_1 with sequence number 0
INFO:qu_net_sim:host_1 received ACK with sequence number 0
INFO:qu_net_sim:QPU_1 received ACK from host_1 with sequence number 0
INFO:qu_net_sim:host_1 received ACK with sequence number 0
INFO:qu_net_sim:QPU_0 received ACK from host_1 with sequence number 0


In [20]:
clock.ticks

4

In [21]:
t1 = controller_host.run_protocol(
    controller_host_protocol_expectation_schedule,
    (q_map, terms))

threads = []

for host in computing_hosts:
    threads.append(host.run_protocol(computing_host_protocol))

t1.join()
for thread in threads:
    thread.join()

INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:2 from QPU_2 to host_1
INFO:qu_net_sim:sending ACK:2 from QPU_0 to host_1
INFO:qu_net_sim:sending ACK:2 from QPU_1 to host_1
INFO:qu_net_sim:QPU_2 received {"QPU_0": [{"name": "REC_HAMILTON", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "hamiltonian": [[-0.04207897647782276, [["Identity", 0]]], [0.17771287465139946, [["PauliZ", 0]]], [0.1777128746513994, [["PauliZ", 1]]], [-0.24274280513140462, [["PauliZ", 2]]], [-0.24274280513140462, [["PauliZ", 3]]]], "layer_end": 4}], "QPU_1": [{"name": "REC_HAMILTON", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_1"], "pre_allocated_qubits": false, "hamiltonian": [[0.17059738328801052, [["PauliZ", 0], ["PauliZ", 1]]], [0.04475014401535161, [["PauliY", 0], ["PauliX", 1], ["PauliX", 2], ["PauliY", 3]]], [-0.04475014401535161, [["PauliY", 0], ["PauliY", 1

In [22]:
clock.ticks

6

In [23]:
t1 = controller_host.run_protocol(
    controller_host_protocol_expectation_collection,
    (q_map, terms))

threads = []

for host in computing_hosts:
    threads.append(host.run_protocol(computing_host_protocol_send_results))

t1.join()
for thread in threads:
    thread.join()

INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:3 from QPU_0 to host_1
INFO:qu_net_sim:sending ACK:3 from QPU_1 to host_1
INFO:qu_net_sim:sending ACK:3 from QPU_2 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "SEND_EXP", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 6}], "QPU_1": [{"name": "SEND_EXP", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_1"], "pre_allocated_qubits": false, "layer_end": 6}], "QPU_2": [{"name": "SEND_EXP", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_2"], "pre_allocated_qubits": false, "layer_end": 6}]} with sequence number 2
INFO:qu_net_sim:QPU_0 sends CLASSICAL to host_1 with sequence 2
INFO:qu_net_sim:host_1 received ACK from QPU_0 with sequence number 2
INFO:qu_net_sim:host_1 received ACK from QPU_1 with sequence number 2
INFO:qu_net_sim:

In [24]:
controller_host.results

{'QPU_0': {'type': 'expectation_value', 'bits': 0.32072833505209875},
 'QPU_1': {'type': 'expectation_value', 'bits': -0.15644186762998807},
 'QPU_2': {'type': 'expectation_value', 'bits': -0.3671466146414246}}

In [25]:
clock.ticks

8

## Optimise

Since the scheduler of the controller host has to be run for each submitted circuit, it is more efficient to put the whole quantum circuit into one Circuit object.

In [26]:
def vqe_circuit(host, q_map, parameters):
    layers = []
    host_id = list(q_map.keys())
    
    # Initialise the qubits on the computing host
    layers = layers + initialisation_operations(q_map)
    
    # Apply the ansatz
    layers = layers + ansatz_operations(q_map, parameters)
    
    # Send the schedults
    layers = layers + dispatch_hamiltonian_schedules(host, q_map)
    
    # Ask for the values
    layers = layers + ask_for_expectation_values(q_map)
    
    circuit = Circuit(q_map, layers)
    return circuit

In [27]:
def controller_host_protocol_schedules(host, q_map, params, terms):
    host.schedule_expectation_terms(terms, q_map)
    
    monolithic_circuit = vqe_circuit(host, q_map, params)

    host.generate_and_send_schedules(monolithic_circuit)

In [28]:
def controller_host_protocol_get_final_results(host, q_map, params):
    host.receive_results()

In [29]:
def computing_host_protocol_receive_schedules(host):
    host.receive_schedule()

In [30]:
def computing_host_protocol_send_final_results(host):
    host.send_results('expectation')

In [35]:
import time
def cost_fn(params):
    params = params.reshape(4, 3)
    
    network = Network.get_instance()
    network.delay = 0
    network.start()

    clock = Clock()

    qutip = QuTipBackend()

    controller_host = ControllerHost(
        host_id="host_1",
        clock=clock,
        backend=qutip
    )

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=2,
        num_qubits_per_host=4)
    
    controller_host.start()

    network.add_hosts(computing_hosts)
    network.add_hosts([controller_host])
    
    #############################################################
    
    t1 = controller_host.run_protocol(controller_host_protocol_schedules, (q_map, params, terms))
    
    threads = []

    for host in computing_hosts:
        threads.append(host.run_protocol(computing_host_protocol_receive_schedules))

    t1.join()
    for thread in threads:
        thread.join()
        
    #############################################################
    
    t1 = controller_host.run_protocol(controller_host_protocol_get_final_results, (q_map, params))
    
    threads = []

    for host in computing_hosts:
        threads.append(host.run_protocol(computing_host_protocol_send_final_results))

    t1.join()
    for thread in threads:
        thread.join()
        
    #############################################################
    
    for host in computing_hosts:
        network.remove_host(host)
    network.remove_host(controller_host)
    
    total_exp = 0
    for host in controller_host.computing_host_ids:
        total_exp += controller_host.results[host]['bits']
    
    return total_exp

In [36]:
# np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

params

tensor([[-0.18798618, -1.8036649 ,  0.82452316],
        [ 1.61004419, -4.95635388, -2.77089489],
        [ 4.79166131, -0.51818046,  2.68272301],
        [ 3.21900415, -0.92974351, -2.87420265]], requires_grad=True)

In [34]:
cost_fn(params)

INFO:qu_net_sim:Host QPU_0 started processing
INFO:qu_net_sim:Host QPU_1 started processing
INFO:qu_net_sim:Host host_1 started processing
INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:1 from QPU_1 to host_1
INFO:qu_net_sim:sending ACK:1 from QPU_0 to host_1
INFO:qu_net_sim:sending ACK:1 from QPU_2 to host_1
INFO:qu_net_sim:host_1 received ACK from QPU_1 with sequence number 0
INFO:qu_net_sim:QPU_1 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGL

INFO:qu_net_sim:QPU_0 sends CLASSICAL to host_1 with sequence 0
INFO:qu_net_sim:host_1 received ACK from QPU_0 with sequence number 0
INFO:qu_net_sim:QPU_2 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "custom_gate", "gate_param": [[[-0.9711634352896334, 0.17974338033198173], [-0.13602207893064863, -0.07766526396436721]], [[0.13602207893064863, -0.07766526396436721], [-0.9711634352896334, -0.17974338033198173]]], "computing_host

1.216788120928726

In [37]:
Logger.DISABLED = True

### SciPy Optimisers

In [38]:
from scipy.optimize import minimize
minimum_bfgs = minimize(cost_fn, params, method="BFGS", tol=1e-4)

minimum_bfgs

      fun: -1.676899183298775
 hess_inv: array([[ 1.00000000e+00,  3.86687204e-07, -3.23915007e-11,
         7.51777289e-13, -1.80505395e-06,  7.51777792e-13,
        -1.84952228e-13, -4.04993969e-06, -1.84952156e-13,
         1.34297677e-12,  3.16075200e-06,  1.35636653e-12],
       [ 3.86687204e-07,  1.59859253e+00,  1.45086657e-06,
        -3.14552177e-07,  4.26225363e-02, -3.14552192e-07,
         3.06335585e-07,  2.40525064e-02,  3.06335623e-07,
        -4.77300984e-08,  7.44482685e-02, -3.58407179e-08],
       [-3.23915007e-11,  1.45086657e-06,  1.00000000e+00,
         3.21834409e-11, -2.66702080e-06,  3.21834408e-11,
        -3.37594282e-11, -5.17880162e-06, -3.37594279e-11,
        -7.98876549e-13,  2.42831305e-06, -7.78995243e-13],
       [ 7.51777289e-13, -3.14552177e-07,  3.21834409e-11,
         1.00000000e+00,  1.75780578e-06,  1.57096617e-12,
        -2.70044146e-12,  4.29978665e-06, -2.70044145e-12,
        -1.65959086e-12, -3.55005303e-06, -1.67302216e-12],
       [-1.

In [39]:
minimum_bfgs.x.reshape(4,3)

array([[-1.87986125e-01, -2.96678931e-05,  8.24523230e-01],
       [ 1.61004423e+00, -6.28313780e+00, -2.77089485e+00],
       [ 4.79166124e+00,  1.91673062e-05,  2.68272294e+00],
       [ 3.21900410e+00,  6.88692856e-05, -2.87420268e+00]])

In [40]:
from scipy.optimize import minimize
minimum_powell = minimize(cost_fn, params, method="Powell", tol=1e-4)

minimum_powell

   direc: array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])
     fun: array(-1.67689919)
 message: 'Optimization terminated successfully.'
    nfev: 324
     nit: 2
  status: 0
 success: True
       x: array([ 1.96448964e+00, -1.85649913e-10,  1.81639254e+00,  4.85675531e+00,
       -6.28318531e+00, -3.88863016e+00,  4.76961204e+00,  1.13051965e-10,
        4.774

In [41]:
minimum_powell.x.reshape(4,3)

array([[ 1.96448964e+00, -1.85649913e-10,  1.81639254e+00],
       [ 4.85675531e+00, -6.28318531e+00, -3.88863016e+00],
       [ 4.76961204e+00,  1.13051965e-10,  4.77457668e+00],
       [ 4.89683156e+00, -3.85706503e-06, -9.50242362e-01]])

In [42]:
from scipy.optimize import minimize
minimum_cobyla = minimize(cost_fn, params, method="COBYLA", tol=1e-4)

minimum_cobyla

     fun: -1.1914135756459883
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 187
  status: 1
 success: True
       x: array([[ 7.20172908e-01, -3.14159394e+00,  1.52745937e+00],
       [-1.27574432e-05, -4.98319157e+00, -2.33235638e+00],
       [ 4.90037061e+00,  4.47879140e-01,  3.14162560e+00],
       [ 3.33259713e+00,  2.55130536e-06, -2.04879492e+00]])

In [43]:
minimum_cobyla.x.reshape(4,3)

array([[ 7.20172908e-01, -3.14159394e+00,  1.52745937e+00],
       [-1.27574432e-05, -4.98319157e+00, -2.33235638e+00],
       [ 4.90037061e+00,  4.47879140e-01,  3.14162560e+00],
       [ 3.33259713e+00,  2.55130536e-06, -2.04879492e+00]])

### Scikit-Quant Optimisers

In [44]:
from skquant.interop.scipy import *

In [45]:
bounds = np.array([-3*np.pi, 3*np.pi])
bounds = np.tile(bounds, (4*3, 1))

flattened_parameters = params.flatten()

In [46]:
budget = 200
minimum_imfil = minimize(cost_fn, flattened_parameters, method=imfil, bounds=bounds, options={'budget' : budget})

minimum_imfil

     fun: -1.1277850814207215
 message: 'completed'
    nfev: 210
  status: 0
 success: True
       x: array([-0.18798618, -2.76678392,  0.82452316,  1.61004419, -6.5710654 ,
       -2.77089489,  4.79166131, -9.42477796,  2.68272301, -1.49338483,
       -9.42477796, -2.87420265])

In [47]:
budget = 100
minimum_bobyqa = minimize(cost_fn, flattened_parameters, method=pybobyqa, bounds=bounds, options={'budget' : budget})

minimum_bobyqa

     fun: -1.4141807413398046
 message: 'completed'
    nfev: 100
  status: 0
 success: True
       x: array([-0.18798618,  0.66250105,  0.82452316,  1.61004419, -6.19847776,
       -2.77089489,  4.79166131,  0.41854572,  2.68272301,  3.21900415,
       -0.02523713, -2.87420265])

In [ ]:
budget = 200
minimum_snobfit = minimize(cost_fn, flattened_parameters, method=snobfit, bounds=bounds, options={'budget' : budget})

minimum_snobfit

### Gradient Descent Optimisers 

In [ ]:
%debug
#max_iterations = 200
#conv_tol = 1e-06

#for n in range(max_iterations):
#    params, prev_energy = opt.step_and_cost(cost_fn, params)
#    energy = cost_fn(params)
#    conv = np.abs(energy - prev_energy)

#    if n % 20 == 0:
#        print('Iteration = {:},  Energy = {:.8f} Ha'.format(n, energy))

#    if conv <= conv_tol:
#        break